# PID interconnects PCB

Import packages:

In [2]:
import pyvisa
import time
import matplotlib.pyplot as plt
import matplotlib.patches as plt_patches
from tqdm import tqdm_notebook
import numpy as np
import h5py

In [3]:
# plot font sizes
factor = 3
SMALL_SIZE = factor*8
MEDIUM_SIZE = factor*10
BIGGER_SIZE = factor*12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

Connect to the serial devices:

In [4]:
rm = pyvisa.ResourceManager()

In [5]:
rm.list_resources()

('ASRL1::INSTR',
 'ASRL7::INSTR',
 'ASRL13::INSTR',
 'ASRL14::INSTR',
 'ASRL15::INSTR')

In [6]:
arduino = rm.open_resource('ASRL13::INSTR')

In [ ]:
arduino.close()

### Arduino for serial communications

Define functions to read and write to the AD9910 registers:

In [15]:
def read_one_register(reg, printing=True):
    # send the read command
    arduino.write_raw(b"1")
    arduino.write_raw(str(reg))
    
    # get Arduino's return text
    return_text = arduino.read(), arduino.read()
    
    if printing:
        for t in return_text:
            print(t, end='')
    
    # convert to list of integers
    return [hex(int(x,2)) for x in return_text[1].strip()[:-1].split(",")]
        
def reset(arduino):
    print(arduino.query('0'))
    
def write_register(reg, data_list, printing=False):
    # send the write command
    arduino.write_raw(b"w")
    arduino.write_raw(str(reg))
    arduino.write_raw(bytes(bytearray(data_list)))
    
    # print Arduino's return text
    return_text = arduino.read(), arduino.read(), arduino.read(), arduino.read(),  arduino.read(),  arduino.read()
    if printing:
        for t in return_text:
            print(t, end='')

Test PLL:

In [9]:
read_one_register(0x02)

Reading from register 2:   
  00011111,00111111,01000000,00000000,


['0x1f', '0x3f', '0x40', '0x0']

In [18]:
write_register(0x02, [0b00011101,0b00111111,0b11000001,0b00110010], printing=True)

Reading from register 2:   
  00011111,00111111,11000000,00000000,
Writing to register 2:   
  00011111,00111111,11000000,00000000,
Reading from register 2:   
  00011111,00111111,11000000,00000000,


Test that the AD9910 outputs anything:

In [13]:
arduino.write("p21 85000000 1000000000 100")
arduino.write("r0")
arduino.write("P0")

(4, <StatusCode.success: 0>)